In [ ]:
import numpy as np
import json
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
from tensorflow import keras
from keras.layers import (
    Input,
    Add,
    Dense,
    ZeroPadding2D,
    BatchNormalization,
    Flatten,
    Conv2D,
    AveragePooling2D,
    MaxPooling2D,
    Dropout
)
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import drive
from tensorflow.keras.layers import Layer

drive.mount('/content/drive')

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = y_train.ravel()
y_test = y_test.ravel()

class Tanh(Layer):
    def call(self, inputs):
        return tf.nn.tanh(inputs)

class ResNet18:
    def __init__(self, num_classes=10, input_shape=(None, None, 3), **kwargs):
        self.num_classes = num_classes
        self.input_shape = input_shape

    def identity_block18(self, x, filters):
        x_skip = x
        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)
        x = Tanh()(x)
        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)
        x = Add()([x, x_skip])
        x = Tanh()(x)
        return x

    def convolutional_block18(self, x, filters):
        x_skip = x
        x = Conv2D(filters, (3, 3), padding='same', strides=(2, 2), kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)
        x = Tanh()(x)
        x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization(axis=3)(x)
        x_skip = Conv2D(filters, (1, 1), strides=(2, 2), kernel_regularizer=l2(1e-4))(x_skip)
        x = Add()([x, x_skip])
        x = Tanh()(x)
        return x

    def build(self, **kwargs):
        x_input = Input(self.input_shape)
        x = ZeroPadding2D((3, 3))(x_input)
        x = Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization()(x)
        x = Tanh()(x)
        x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
        filter_sizes = [64, 128, 256, 512]
        block_layers = [2, 2, 2, 2]

        for i in range(4):
            filters = filter_sizes[i]
            if i == 0:
                for _ in range(block_layers[i]):
                    x = self.identity_block18(x, filters)
            else:
                x = self.convolutional_block18(x, filters)
                for _ in range(block_layers[i] - 1):
                    x = self.identity_block18(x, filters)

        x = AveragePooling2D((2, 2), padding='same')(x)
        x = Flatten()(x)
        x = Dropout(0.5)(x)
        x = Dense(self.num_classes, activation='softmax', kernel_regularizer=l2(1e-4))(x)

        return Model(inputs=x_input, outputs=x, name="ResNet18")

rn = ResNet18(input_shape=(32, 32, 3))
model = rn.build()
model.summary()

model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

earlystop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

learning = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    verbose=1,
    callbacks=[earlystop, reduce_lr]
)

loss_train = learning.history["loss"]
loss_val = learning.history["val_loss"]
acc_train = learning.history["sparse_categorical_accuracy"]
acc_val = learning.history["val_sparse_categorical_accuracy"]

y_pred = [np.argmax(arr) for arr in model.predict(X_test)]
prf_score = precision_recall_fscore_support(y_test, y_pred, average="weighted")
score = model.evaluate(X_test, y_test)

performance = {
    'loss_train': loss_train,
    'loss_val': loss_val,
    'acc_train': acc_train,
    'acc_val': acc_val,
    'prf_score': prf_score,
    'score': score
}

performance_file_path = '/content/drive/My Drive/Colab Notebooks/tanh_performance.json'
with open(performance_file_path, 'w') as f:
    json.dump(performance, f)

print("Performance saved to:", performance_file_path)

Mounted at /content/drive
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "ResNet18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 38, 38, 3) │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 19, 19,    │      9,472 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 19, 19,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tanh (Tanh)         │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 10, 10,    │          0 │ tanh[0][0]        │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 10, 10,    │     36,928 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 10,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tanh_1 (Tanh)       │ (None, 10, 10,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 10, 10,    │     36,928 │ tanh_1[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 10,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 10, 10,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tanh_2 (Tanh)       │ (None, 10, 10,    │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 10, 10,    │     36,928 │ tanh_2[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 10,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tanh_3 (Tanh)       │ (None, 10, 10,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 10, 10,    │     36,928 │ tanh_3[0][0]    

 Total params: 11,192,458 (42.70 MB)

 Trainable params: 11,184,650 (42.67 MB)

 Non-trainable params: 7,808 (30.50 KB)

Epoch 1/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 75s 32ms/step - loss: 2.1644 - sparse_categorical_accuracy: 0.3588 - val_loss: 2.0931 - val_sparse_categorical_accuracy: 0.4078 - learning_rate: 0.0010
Epoch 2/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 49s 19ms/step - loss: 1.5754 - sparse_categorical_accuracy: 0.5322 - val_loss: 2.2553 - val_sparse_categorical_accuracy: 0.3925 - learning_rate: 0.0010
Epoch 3/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 42s 20ms/step - loss: 1.4351 - sparse_categorical_accuracy: 0.5809 - val_loss: 1.6626 - val_sparse_categorical_accuracy: 0.5234 - learning_rate: 0.0010
Epoch 4/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - loss: 1.3448 - sparse_categorical_accuracy: 0.6219 - val_loss: 1.3954 - val_sparse_categorical_accuracy: 0.6034 - learning_rate: 0.0010
Epoch 5/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - loss: 1.2924 - sparse_categorical_accuracy: 0.6444 - val_loss: 1.4969 - val_sparse_categorical_accuracy: 0.5827 - learning_rate: 0.0010
Epoch 6/30
1563/1563 ━━━━━━━━━